# *Travelling Salesman Problem* (TSP)

Desenvolvimento de Algoritmo Genético (GA) e Otimização por Enxame de Partículas (PSO) para solucionar o problema do Caixeiro Viajante feita durante a disciplina de Computação Bioinspirada pela UNIFESP.

Feito por Gabriel Angelo Cabral Neves.





# Tabela de distância entre as cidades

In [6]:
import pandas as pd

def read_table_of_distances(fp):
    """
    Open a .csv file of the distances between each city pair

    :param fp: File path
    :return: Data frame of the distances
    """
    df = pd.read_csv(fp, index_col=0)

    # Check if there is a NaN value
    if not df.isnull().values.any():
        return df

    # Populate data frame if only half of it is complete
    for col in df:
        for lin in df[col].keys():
            df[lin][col] = df[col][lin]

    return df
  
print(read_table_of_distances('distances.csv'))

    A   B   C   D   E   F   G   H
A   -  42  61  30  17  82  31  11
B  42   -  14  87  28  70  19  33
C  61  14   -  20  87  28   8  29
D  30  87  20   -  34  33  91  10
E  17  28  87  34   -  41  34  82
F  82  70  28  33  41   -  19  32
G  31  19   8  91  34  19   -  59
H  11  33  29  10  82  32  59   -


# Função de aptidão

In [9]:
def distance_of_route(route, table_of_distances):
    """
    Calculates the total distance traveled of a route

    :param route: Desired route
    :param table_of_distances: Table of distances containing each city pair
    :return: Total distance traveled
    """
    no_of_cities = len(route)
    total_distance = 0
    for city in range(no_of_cities):
        current_city = route[city]
        next_city = route[city + 1] if city < (no_of_cities - 1) else route[0]
        total_distance += int(table_of_distances[current_city][next_city])
        
    return total_distance


# Genetic Algorithm (GA)

Para representarmos a solução, que podemos chamar de cromossomo, a classe Chromossome foi criada, possuindo os atributos de rota e aptidão, além de possuir um método que faz a mutação de um gene trocando duas cidades da rota de lugar.

In [3]:
from random import sample
import math

class Chromosome:
    def __init__(self, route):
        self.route = route
        self.fitness = math.inf

    def mutate(self):
        city1, city2 = sample(range(1, len(self.route)), 2)
        self.route[city1], self.route[city2] = self.route[city2], self.route[city1]

A função de seleção, que descreve o método de torneio para a escolha dos pais que gererão os filhos, e também a que descreve o operador genético de *crossover* podem ser encontradas logo abaixo. 

A primeira utiliza a aptidão de N indivíduos selecionados aleatoriamente para ordená-los e selecionar o melhor dentre eles para se tornar o pai de um filho que será gerado posteriormente.

Já o operador genético possui um comportamento um pouco mais complexo. É escolhido aleatoriamente um intervalo dos pais que será passado automaticamente aos filhos, e as demais cidades serão pegadas do outro pai na ordem em que elas aparecem na rota, evitando repetir aquelas que já foram escolhidas.

In [4]:
def tournament(parent_list, n):
    """
    Make a crossover between two parents of solutions of TSP. Basically an interval is generated randomly and each child
    receives a slice of one of the parents based on these values, then it is completed with the missing cities in the
    order they appear in the other parent.

    :param parent1: Parent one
    :param parent2: Parent two
    :return: Two chromosomes generated from the crossover of parent1 and parent2
    """
    candidates = sample(parent_list, n)
    return sorted(candidates, key=lambda chromosome: chromosome.fitness, reverse=False)[0]

def crossover(parent1, parent2):
    """
    Make a crossover between two parents of solutions of TSP. Basically an interval is generated randomly and each child
    receives a slice of one of the parents based on these values, then it is completed with the missing cities in the
    order they appear in the other parent.

    :param parent1: Parent one
    :param parent2: Parent two
    :return: Two chromosomes generated from the crossover of parent1 and parent2
    """
    length = len(parent1.route)
    child1_route = [None] * length
    child2_route = [None] * length

    indexes = sample(range(length), 2)
    start, end = min(indexes), max(indexes)
    end += 1
    child1_route[start:end], child2_route[start:end] = parent2.route[start:end], parent1.route[start:end]

    parent_order_index = 0
    for child_city in range(length):
        if child1_route[child_city] is None:
            while parent1.route[parent_order_index] in child1_route:
                parent_order_index += 1
            child1_route[child_city] = parent1.route[parent_order_index]

    parent_order_index = 0
    for child_city in range(length):
        if child2_route[child_city] is None:
            while parent2.route[parent_order_index] in child2_route:
                parent_order_index += 1
            child2_route[child_city] = parent2.route[parent_order_index]

    return Chromosome(child1_route), Chromosome(child2_route)


O Algoritmo Genético (GA) em si, possui algumas etapas para sua utilização, sendo primeiro a sua configuração de parâmetros, inicialização e avaliação da população, e o processo de evolução que repetirá pelas gerações desejadas.


In [11]:
from random import random

class GA:
    def __init__(self, table_of_distances, population_size, generations, mutation_rate, elitism, tournament_size,
                 fitness_function):
        """
        Genetic Algorithm to solve TSP

        :param table_of_distances: Distances of each city pair
        :param population_size: Number of individuals in population
        :param generations: Maximum number of generations
        :param mutation_rate: Probability of an individual of the population to mutate
        :param elitism: Number of the best individuals to be carried over to the next generation
        :param tournament_size: Size of the tournament selection
        :param fitness_function: Function to evaluate each individual of the population
        """
        self.table_of_distances = table_of_distances
        self.generations = generations
        self.population_size = population_size
        self.population = []
        self.__initialize_population()
        self.ranking = []
        self.elitism = elitism
        self.tournament_size = tournament_size
        self.mutation_rate = mutation_rate
        self.fitness_function = fitness_function

    def __initialize_population(self):
        cities = list(self.table_of_distances.index.values)
        for individual in range(self.population_size):
            initial_route = sample(cities, len(cities))
            self.population.append(Chromosome(initial_route))

    def evaluate_population(self):
        self.ranking = []
        for chromosome in self.population:
            chromosome.fitness = self.fitness_function(chromosome.route, self.table_of_distances)
            if not self.ranking:
                self.ranking.append(chromosome)
                continue
            for position in self.ranking:
                if chromosome.fitness < position.fitness:
                    self.ranking.insert(self.ranking.index(position), chromosome)
                    break
            else:
                self.ranking.append(chromosome)

    def generate_new_population(self):
        new_population = []

        # Elitism warranty
        new_population += self.ranking[:self.elitism]

        # Parent selection
        parent_list = []
        for _ in range(self.population_size - self.elitism):
            parent = tournament(self.ranking, self.tournament_size)
            parent_list.append(parent)

        # Crossover
        length = len(parent_list)
        for i in range(math.ceil(length / 2)):
            parent1 = parent_list[i]
            parent2 = parent_list[length - i - 1] if i != length - i - 1 else sample(parent_list[:i - 1] +
                                                                                     parent_list[i + 1:], 1)[0]
            child1, child2 = crossover(parent1, parent2)
            new_population.append(child1)
            new_population.append(child2)

        # Mutation
        for individual in new_population[self.elitism:]:
            probability = random()
            if probability <= self.mutation_rate:
                individual.mutate()
              
        self.population = new_population

    def evolve(self):
        for _ in range(self.generations):
            self.evaluate_population()
            self.generate_new_population()
        print('Best route is', self.ranking[0].route, 'with a distance equal to', self.ranking[0].fitness)

## Configuração de parâmetros e avaliação do algoritmo genético

In [14]:
cities_distance = read_table_of_distances('distances.csv')
population = 50
generations = 1000
mutation_rate = 0.01
elitism = 5
tournament_selection = 3
fitness_func = distance_of_route
for _ in range(15):
    ga = GA(cities_distance, population, generations, mutation_rate, elitism, tournament_selection, fitness_func)
    ga.evolve()

Best route is ['E', 'A', 'H', 'D', 'F', 'G', 'C', 'B'] with a distance equal to 140
Best route is ['F', 'D', 'H', 'A', 'E', 'B', 'C', 'G'] with a distance equal to 140
Best route is ['A', 'E', 'F', 'G', 'B', 'C', 'D', 'H'] with a distance equal to 151
Best route is ['A', 'H', 'D', 'F', 'G', 'C', 'B', 'E'] with a distance equal to 140
Best route is ['F', 'G', 'B', 'C', 'D', 'H', 'A', 'E'] with a distance equal to 151
Best route is ['E', 'A', 'H', 'D', 'F', 'G', 'C', 'B'] with a distance equal to 140
Best route is ['E', 'A', 'H', 'D', 'C', 'B', 'G', 'F'] with a distance equal to 151
Best route is ['E', 'A', 'H', 'D', 'F', 'G', 'C', 'B'] with a distance equal to 140
Best route is ['B', 'E', 'A', 'G', 'F', 'H', 'D', 'C'] with a distance equal to 171
Best route is ['B', 'C', 'G', 'F', 'D', 'H', 'A', 'E'] with a distance equal to 140
Best route is ['F', 'D', 'H', 'A', 'E', 'B', 'C', 'G'] with a distance equal to 140
Best route is ['B', 'C', 'G', 'F', 'E', 'A', 'D', 'H'] with a distance equal

# *Particle Swarm Optimization* (PSO)

Para representar as párticulas (soluções) do PSO, foi desenvolvido a classe *Particle* que possui como atributos a rota (posição), sequência de troca (velocidade), a aptidão atual, a melhor aptidão encontrada e a rota referente a essa melhor aptdidão.

Lembrando que a atualização da posição e da velocidade devem seguir as seguintes equações, onde serão feitas algumas definições para que seja possível aplicá-las no TSP.

> $v(t) = v(t-1) + Φ_{1} (x_{global} - x(t-1)) + Φ_{2} (x_{local} - x(t-1))$ \\
> $x(t) = x(t-1) + v(t-1)$ \\
>
> **Sendo:** \\
> $Φ_{1}$ = Coeficiente da melhor solução global \\
> $x_{global} $ = Melhor solução global \\
> $Φ_{2}$ = Coeficiente da melhor solução local \\
> $x_{local} $ = Melhor solução local

Como as rotas serão tratadas como a posição, deve-se definir qual o significado da operação de subtração entre duas rotas, e consequentemente definir a velocidade e como ela altera a posição em sua atualização.

Podemos tratar a operação $SS = A - B$ entre duas rotas como **a sequência de trocas entre as cidades da rota $B$ para que ela seja igual a rota $A$**. Dessa forma é possível chegar que $A = B + SS$, e traçando um parelelo entre a equação da posição define-se a velocidade como uma determinada sequência de trocas a ser aplicada na posição para sua atualização.

Portanto foi criada a classe *SwapSequence* que representa a velocidade de uma párticula, assim como o resultado da operação de subtração entre duas rotas.

In [16]:
from random import randint

class SwapSequence:
    def __init__(self):
        self.queue = []

    def enqueue(self, exchange_tuple):
        self.queue.append(exchange_tuple)

    def dequeue(self):
        return self.queue.pop(0)

    def enqueue_ss(self, ss_queue):
        self.queue.extend(ss_queue)

class Particle:
    def __init__(self, route):
        """
        Individual of the swarm

        :param route: Initial route of the particle
        """
        self.route = route  # Particle position
        self.swap_sequence = SwapSequence()  # Particle velocity
        self.__generate_initial_swap_sequence()  # Initial velocity
        self.fitness = math.inf
        self.my_best_fitness = math.inf
        self.my_best_route = []

    def __generate_initial_swap_sequence(self):
        cities_number = len(self.route)
        swap_no = randint(0, cities_number)
        for swap in range(swap_no):
            city_i = randint(0, cities_number - 1)
            city_j = randint(0, cities_number - 1)
            self.swap_sequence.enqueue((city_i, city_j))

def swap_cities(route, i, j):
    """
    Swap element of index i with element of index j in array route

    :param route: Array of elements to be swapped
    :param i: Index of the first element
    :param j: Index of the second element
    """
    route[i], route[j] = route[j], route[i]


def routes_subtraction(route_one, route_two):
    """
    Generate a SwapSequence for the route_two to become the route_one (SS = A - B => B + SS = A)

    :param route_one: Left operand of the subtraction
    :param route_two: Right operand of the subtraction
    :return: SwapSequence for route_two
    """
    swappable_route_two = route_two.copy()
    result_ss = SwapSequence()
    length = len(route_one)
    index = 0
    while index < length:
        if route_one[index] != swappable_route_two[index]:
            swap_index = swappable_route_two.index(route_one[index])
            result_ss.enqueue((index, swap_index))
            swap_cities(swappable_route_two, index, swap_index)
        index += 1

    return result_ss

A classe PSO possui algumas fases assim como o GA, onde incialmente se é configurado os parâmetros desejados, a geração e avaliação da população inicial de párticulas e a evolução desse algoritmo. Ressaltando que os parâmetros $Φ$ representam o quanto de sequência de trocas serão mantidas referentes as diferenças entre a posição atual e a global e local. 



In [17]:
class PSO:
    def __init__(self, table_of_distances: pd.DataFrame, population_size, max_epochs, fitness_function, phi_one, phi_two):
        """
        Particle Swarm Optimization to solve TSP

        :param table_of_distances: Distances of each city pair
        :param population_size: Number of individuals in swarm
        :param max_epochs: Max number of epochs
        :param fitness_function: Function to evaluate each particle of the swarm
        :param phi_one: Probability of the particle to go in direction of the GLOBAL best
        :param phi_two: Probability of the particle to go in direction of the LOCAL best
        """
        self.table_of_distances = table_of_distances
        self.max_epochs = max_epochs
        self.population_size = population_size
        self.population = []
        self.__initialize_population()
        self.fitness_function = fitness_function
        self.global_best_solution_route = []
        self.global_best_solution_fitness = math.inf
        self.phi_one = phi_one
        self.phi_two = phi_two

    def __initialize_population(self):
        cities = list(self.table_of_distances.index.values)
        for individual in range(self.population_size):
            initial_route = sample(cities, len(cities))
            self.population.append(Particle(initial_route))
        self.global_best_solution_route = self.population[0].route
        self.global_best_solution_fitness = self.population[0].fitness

    def evaluate_population(self):
        for particle in self.population:
            fitness = self.fitness_function(particle.route, self.table_of_distances)
            particle.fitness = fitness
            if fitness < particle.my_best_fitness:
                particle.my_best_fitness, particle.my_best_route = fitness, particle.route
            if fitness < self.global_best_solution_fitness:
                self.global_best_solution_route, self.global_best_solution_fitness = particle.route.copy(), particle.fitness

    def update_population(self):
        for particle in self.population:
            # Updating velocity
            # V(t) = V(t-1) + phi_one * (best_global - current_route) + phi_two * (best_local - current_route)

            # phi_one * (best_global - current_route)
            global_subtraction = routes_subtraction(self.global_best_solution_route, particle.route)
            aux_global_subtraction_queue = global_subtraction.queue.copy()
            for swap in global_subtraction.queue:
                probability = random()
                if probability <= self.phi_one:
                    aux_global_subtraction_queue.remove(swap)
            global_subtraction.queue = aux_global_subtraction_queue

            # phi_two * (best_local - current_route)
            local_subtraction = routes_subtraction(particle.my_best_route, particle.route)
            aux_local_subtraction_queue = local_subtraction.queue.copy()
            for swap in local_subtraction.queue:
                probability = random()
                if probability <= self.phi_two:
                    aux_local_subtraction_queue.remove(swap)
            local_subtraction.queue = aux_local_subtraction_queue

            # Update position
            # X(t) = X(t-1) + V(t-1)
            for swap in particle.swap_sequence.queue:
                swap_cities(particle.route, swap[0], swap[1])

            # Update V(t)
            particle.swap_sequence.enqueue_ss(global_subtraction.queue)
            particle.swap_sequence.enqueue_ss(local_subtraction.queue)

    def evolve(self):
        for epoch in range(self.max_epochs):
            self.evaluate_population()
            self.update_population()
        print('Best route is', self.global_best_solution_route, 'with a distance equal to',
              self.global_best_solution_fitness)

## Configuração de parâmetros e avaliação do PSO

In [19]:
cities_distance = read_table_of_distances('distances.csv')
population = 30
epochs = 1000
fitness_func = distance_of_route
global_probability = 0.4
local_probability = 0.6
for _ in range(15):
    pso = PSO(cities_distance, population, epochs, fitness_func, global_probability, local_probability)
    pso.evolve()


Best route is ['H', 'A', 'E', 'B', 'C', 'G', 'F', 'D'] with a distance equal to 140
Best route is ['F', 'G', 'B', 'C', 'D', 'H', 'A', 'E'] with a distance equal to 151
Best route is ['A', 'E', 'B', 'C', 'G', 'F', 'D', 'H'] with a distance equal to 140
Best route is ['F', 'G', 'C', 'B', 'E', 'A', 'H', 'D'] with a distance equal to 140
Best route is ['D', 'H', 'A', 'E', 'B', 'C', 'G', 'F'] with a distance equal to 140
Best route is ['F', 'D', 'H', 'A', 'E', 'B', 'C', 'G'] with a distance equal to 140
Best route is ['D', 'F', 'G', 'C', 'B', 'E', 'A', 'H'] with a distance equal to 140
Best route is ['D', 'C', 'G', 'B', 'E', 'A', 'H', 'F'] with a distance equal to 168
Best route is ['E', 'A', 'H', 'D', 'F', 'G', 'C', 'B'] with a distance equal to 140
Best route is ['C', 'G', 'F', 'D', 'H', 'A', 'E', 'B'] with a distance equal to 140
Best route is ['F', 'G', 'C', 'B', 'E', 'A', 'H', 'D'] with a distance equal to 140
Best route is ['E', 'F', 'G', 'B', 'C', 'D', 'H', 'A'] with a distance equal